# 🎙️ Thu Voice cho Teacher Scripts

Thu âm giọng nói của bạn cho từng script, sau đó push lên Git.

In [ ]:
# @title 1️⃣ Cấu hình
GIT_TOKEN = "" # @param {type:"string"}
GIT_REPO = "nmnhut-it/english-learning-app" # @param {type:"string"}
GIT_BRANCH = "main" # @param {type:"string"}
MD_FILE = "v2/data/voice-lectures/g6/unit-07/getting-started.md" # @param {type:"string"}

In [ ]:
# @title 2️⃣ Clone repo & Parse scripts
import os, re

!rm -rf repo
!git clone https://{GIT_TOKEN}@github.com/{GIT_REPO}.git repo --branch {GIT_BRANCH} --depth 1 -q
os.chdir('repo')

with open(MD_FILE, 'r') as f:
    content = f.read()

scripts = []
for m in re.finditer(r'<teacher_script([^>]*)>([\s\S]*?)</teacher_script>', content, re.I):
    attrs, text = m.group(1), m.group(2).strip()
    if text and 'href=' not in attrs:
        scripts.append({'idx': len(scripts), 'text': text, 'attrs': attrs, 'match': m.group(0)})

print(f"📝 Tìm thấy {len(scripts)} scripts cần thu âm")

# Create audio folder
audio_dir = os.path.dirname(MD_FILE) + '/audio'
os.makedirs(audio_dir, exist_ok=True)

In [ ]:
# @title 3️⃣ Thu âm từng script
from google.colab import output
from IPython.display import display, HTML, clear_output
import base64

current_idx = 0
recordings = {}

def save_audio(idx, audio_b64):
    audio_bytes = base64.b64decode(audio_b64.split(',')[1])
    path = f"{audio_dir}/ts_{idx:03d}.mp3"
    with open(path, 'wb') as f:
        f.write(audio_bytes)
    recordings[idx] = path
    print(f"✅ Saved: {path}")

output.register_callback('notebook.save_audio', save_audio)

def show_recorder(idx):
    if idx >= len(scripts):
        print("🎉 Đã thu xong tất cả! Chạy cell tiếp theo để push.")
        return
    
    s = scripts[idx]
    html = f'''
    <div style="font-family:sans-serif;max-width:600px;margin:20px auto;padding:20px;background:#fff;border-radius:12px;box-shadow:0 2px 10px rgba(0,0,0,.1)">
        <div style="color:#666;font-size:14px">Script {idx+1} / {len(scripts)}</div>
        <div style="background:linear-gradient(90deg,#2563eb {int((idx+1)/len(scripts)*100)}%,#e5e7eb {int((idx+1)/len(scripts)*100)}%);height:6px;border-radius:3px;margin:10px 0"></div>
        <div style="background:#eff6ff;border-left:4px solid #2563eb;padding:16px;border-radius:0 8px 8px 0;margin:16px 0;font-size:18px;line-height:1.6">{s["text"]}</div>
        <div style="text-align:center">
            <button id="recBtn" onclick="toggleRec()" style="width:80px;height:80px;border-radius:50%;border:none;background:#2563eb;color:#fff;font-size:32px;cursor:pointer">🎤</button>
            <div id="status" style="margin:12px 0;color:#666">Bấm 🎤 để thu</div>
        </div>
        <audio id="player" controls style="width:100%;display:none;margin:12px 0"></audio>
        <div id="btns" style="display:none;text-align:center">
            <button onclick="reRecord()" style="padding:10px 20px;margin:4px;border:none;background:#e5e7eb;border-radius:6px;cursor:pointer">🔄 Thu lại</button>
            <button onclick="acceptNext()" style="padding:10px 20px;margin:4px;border:none;background:#10b981;color:#fff;border-radius:6px;cursor:pointer">✅ OK, Tiếp</button>
        </div>
        <div style="text-align:center;margin-top:16px">
            <button onclick="skipNext()" style="padding:8px 16px;border:none;background:#f3f4f6;border-radius:6px;cursor:pointer;color:#666">Bỏ qua ➡️</button>
        </div>
    </div>
    <script>
    let mediaRecorder, chunks=[], recording=false, audioBlob;
    async function toggleRec() {{
        const btn = document.getElementById('recBtn');
        const status = document.getElementById('status');
        if (recording) {{
            mediaRecorder.stop();
            btn.textContent = '🎤';
            btn.style.background = '#2563eb';
            status.textContent = 'Đang xử lý...';
            recording = false;
        }} else {{
            const stream = await navigator.mediaDevices.getUserMedia({{audio:true}});
            mediaRecorder = new MediaRecorder(stream);
            chunks = [];
            mediaRecorder.ondataavailable = e => chunks.push(e.data);
            mediaRecorder.onstop = () => {{
                stream.getTracks().forEach(t=>t.stop());
                audioBlob = new Blob(chunks, {{type:'audio/mp3'}});
                const url = URL.createObjectURL(audioBlob);
                const player = document.getElementById('player');
                player.src = url;
                player.style.display = 'block';
                document.getElementById('btns').style.display = 'block';
                status.textContent = 'Nghe lại rồi chọn OK hoặc Thu lại';
            }};
            mediaRecorder.start();
            btn.textContent = '⏹️';
            btn.style.background = '#ef4444';
            status.textContent = '🔴 Đang thu...';
            document.getElementById('player').style.display = 'none';
            document.getElementById('btns').style.display = 'none';
            recording = true;
        }}
    }}
    function reRecord() {{
        document.getElementById('player').style.display = 'none';
        document.getElementById('btns').style.display = 'none';
        document.getElementById('status').textContent = 'Bấm 🎤 để thu lại';
    }}
    function acceptNext() {{
        const reader = new FileReader();
        reader.onload = () => {{
            google.colab.kernel.invokeFunction('notebook.save_audio', [{idx}, reader.result], {{}});
            google.colab.kernel.invokeFunction('notebook.next_script', [], {{}});
        }};
        reader.readAsDataURL(audioBlob);
    }}
    function skipNext() {{
        google.colab.kernel.invokeFunction('notebook.next_script', [], {{}});
    }}
    </script>
    '''
    clear_output(wait=True)
    display(HTML(html))

def next_script():
    global current_idx
    current_idx += 1
    show_recorder(current_idx)

output.register_callback('notebook.next_script', next_script)

# Start
show_recorder(0)

In [ ]:
# @title 4️⃣ Update markdown & Push
print(f"📊 Đã thu {len(recordings)} / {len(scripts)} scripts\n")

# Update markdown
updated = content
for idx, path in recordings.items():
    s = scripts[idx]
    filename = os.path.basename(path)
    new_tag = f'<teacher_script {s["attrs"]} href="audio/{filename}">{s["text"]}</teacher_script>'
    updated = updated.replace(s['match'], new_tag)

with open(MD_FILE, 'w') as f:
    f.write(updated)

print(f"✅ Updated {MD_FILE}")

# Push
!git config user.email "colab@local"
!git config user.name "Colab Voice Recorder"
!git add .
!git commit -m "Add voice recordings for {os.path.basename(MD_FILE)}"
!git push
print("\n🎉 Done! Pushed to Git.")